In [15]:
import os, json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import random
import fsspec
import gcsfs
import io

from mplsoccer import Pitch

import umap
import hdbscan
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
pd.set_option('display.max_columns', None)

In [11]:
BUCKET_NAME = "arthur_tmp"
BASE_GCS_PATH = f"gs://{BUCKET_NAME}"

DYNAMIC_DIR = f"{BASE_GCS_PATH}/dynamic"
META_DIR = f"{BASE_GCS_PATH}/meta"
TRACKING_DIR = f"{BASE_GCS_PATH}/tracking"
POSSESSION_DIR = f"{BASE_GCS_PATH}/possession"
PITCH_CONTROL_DIR = f"{BASE_GCS_PATH}/pitch_control"
CLUSTERING_DIR = f"{BUCKET_NAME}/clustering"

# Pitch constants (SkillCorner → 105x68, origin at left-bottom after rescale)
X_MIN, X_MAX = -52, 52
Y_MIN, Y_MAX = -34, 34
PITCH_LENGTH, PITCH_WIDTH = 105.0, 68.0

In [4]:
def prepare_clustering_features():
    """
    Transforme la liste de dictionnaires en matrice de features pour le clustering.
    Lit les informations contextuelles (player, équipe, minute, etc.)
    directement depuis le fichier POSSESSION_DIR/<match_id>.parquet.

    Les frames contenant des NaN dans 'player_position', 
    'distance_to_nearest_teammate' ou 'distance_to_nearest_opponent' 
    sont exclues des résultats.

    Colonnes ajoutées dans meta :
        - match_id
        - player_id
        - player_name
        - player_team
        - opponent_team
        - minute
    """

    fs = gcsfs.GCSFileSystem()
    feature_vectors = []
    meta_info = []
    frames_inputs = []

    pitch_control_files = fs.glob(f"{PITCH_CONTROL_DIR}/*.npz")
    if not pitch_control_files:
        print(f"⚠️ Aucun fichier trouvé dans {PITCH_CONTROL_DIR}")
        return None, None, None

    for gcs_path in sorted(pitch_control_files):
        match_id = gcs_path.split("/")[-1].replace(".npz", "")

        # --- Lecture du fichier .npz ---
        try:
            with fs.open(gcs_path, "rb") as f:
                data = np.load(f, allow_pickle=True)
                results = data["results"].tolist()
        except Exception as e:
            print(f"⚠️ Erreur lecture {gcs_path} : {e}")
            continue

        if not results:
            continue

        # --- Lecture du fichier POSSESSION correspondant ---
        possession_path = f"{POSSESSION_DIR}/{match_id}.parquet"
        try:
            with fs.open(possession_path, "rb") as fp:
                possession_df = pd.read_parquet(fp)
        except Exception as e:
            print(f"⚠️ Impossible de lire {possession_path} : {e}")
            possession_df = None

        # --- Préparer les noms d'équipes (pour déduction adversaire) ---
        all_teams = (
            possession_df["team_name"].unique().tolist()
            if possession_df is not None and "team_name" in possession_df.columns
            else []
        )

        # --- Itération sur les résultats valides ---
        for item in results:
            # Exclusion si NaN critiques
            player_position = item.get("player_position", (np.nan, np.nan))
            if (
                np.isnan(player_position[0]) or np.isnan(player_position[1]) or
                np.isnan(item.get("distance_to_nearest_teammate", np.nan)) or
                np.isnan(item.get("distance_to_nearest_opponent", np.nan))
            ):
                continue  # on ignore cette frame

            player_id = item.get("player_id")
            frame = item.get("frame")

            # (1) Flatten de la carte de Pitch Control
            pitch_flat = item["pitch_control_map"].flatten()

            # (2) Features contextuelles
            defensive_lines = np.array(item["defensive_lines"], dtype=float)
            defensive_lines = np.pad(
                defensive_lines, (0, 3 - len(defensive_lines)),
                mode='constant', constant_values=np.nan
            )

            ball_x, ball_y = item["ball_position"]
            in_possession = 1.0 if item["in_possession"] else 0.0

            context_features = np.array([
                *defensive_lines,
                ball_x,
                ball_y,
                in_possession,
                item["distance_to_ball"],
                item["distance_to_nearest_teammate"],
                item["distance_to_nearest_opponent"]
            ], dtype=float)

            full_vector = np.concatenate([pitch_flat, context_features])
            feature_vectors.append(full_vector)
            frames_inputs.append(item)

            # (3) Récupération des infos depuis le parquet possession
            if possession_df is not None:
                row = possession_df[
                    (possession_df["frame"] == frame) &
                    (possession_df["player_id"] == player_id)
                ]
                if not row.empty:
                    row = row.iloc[0]
                    player_name = row.get("player_short_name", None)
                    player_team = row.get("team_name", None)
                    minute = None
                    time_val = row.get("time", None)
                    if isinstance(time_val, str) and len(time_val) >= 5:
                        try:
                            minute = int(time_val.split(":")[1])
                        except Exception:
                            minute = None

                    # Déduire l’adversaire
                    opponent_team = None
                    if all_teams and player_team in all_teams:
                        opponent_team = [t for t in all_teams if t != player_team]
                        opponent_team = opponent_team[0] if opponent_team else None
                else:
                    player_name = None
                    player_team = None
                    opponent_team = None
                    minute = None
            else:
                player_name = None
                player_team = None
                opponent_team = None
                minute = None

            # (4) Ajouter au meta_info
            meta_info.append({
                "match_id": match_id,
                "frame": frame,
                "player_id": player_id,
                "player_position_role": item.get("player_position_role", None),
                "player_name": player_name,
                "player_team": player_team,
                "opponent_team": opponent_team,
                "minute": minute
            })

    if not feature_vectors:
        print("⚠️ Aucun vecteur de features généré (fichiers vides ?)")
        return None, None, None

    # --- Matrice finale des features ---
    X = np.vstack(feature_vectors)

    # --- Gestion des NaN restants (remplacés par moyenne colonne) ---
    nan_mask = np.isnan(X)
    if np.any(nan_mask):
        col_means = np.nanmean(X, axis=0)
        inds = np.where(nan_mask)
        X[inds] = np.take(col_means, inds[1])

    meta = pd.DataFrame(meta_info)
    return X, meta, frames_inputs

In [5]:
%%time

X, meta, frames_inputs = prepare_clustering_features()

CPU times: user 54.3 s, sys: 4.94 s, total: 59.3 s
Wall time: 7min 56s


In [13]:
fs = gcsfs.GCSFileSystem()
fs.mkdirs(CLUSTERING_DIR, exist_ok=True)

X_path = f"{CLUSTERING_DIR}/X_features_center_forward.npz"
with fs.open(X_path, "wb") as f:
    np.savez_compressed(f, X=X)

print(f"✅ Matrice X sauvegardée sur GCS : {X_path} (shape={X.shape})")

✅ Matrice X sauvegardée sur GCS : arthur_tmp/clustering/X_features_center_forward.npz (shape=(78865, 1609))


In [14]:
meta_path = f"{CLUSTERING_DIR}/meta_frames_center_forward_w_labels.parquet"
with fs.open(meta_path, "wb") as f:
    meta.to_parquet(f, index=False)
    
print(f"✅ DataFrame meta sauvegardé sur GCS : {meta_path} (shape={meta.shape})")

✅ DataFrame meta sauvegardé sur GCS : arthur_tmp/clustering/meta_frames_center_forward_w_labels.parquet (shape=(78865, 8))


In [61]:
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021

In [6]:
meta

,match_id,frame,player_id,player_position_role,player_name,player_team,opponent_team,minute
0,1650385,1625,1498,Center Forward,Bruno Fernandes,Manchester United,Fulham,2
1,1650385,1648,1498,Center Forward,Bruno Fernandes,Manchester United,Fulham,2
2,1650385,1692,1498,Center Forward,Bruno Fernandes,Manchester United,Fulham,2
3,1650385,1699,1498,Center Forward,Bruno Fernandes,Manchester United,Fulham,2
4,1650385,1728,1498,Center Forward,Bruno Fernandes,Manchester United,Fulham,2
...,...,...,...,...,...,...,...,...
78860,2018580,54746,13368,Center Forward,Y. Wissa,Brentford FC,Wolverhampton Wanderers,29
78861,2018580,56507,13368,Center Forward,Y. Wissa,Brentford FC,Wolverhampton Wanderers,32
78862,2018580,56545,13368,Center Forward,Y. Wissa,Brentford FC,Wolverhampton Wanderers,32
78863,2018580,56572,13368,Center Forward,Y. Wissa,Brentford FC,Wolverhampton Wanderers,32


In [7]:
meta.isna().sum()

match_id                0
frame                   0
player_id               0
player_position_role    0
player_name             0
player_team             0
opponent_team           0
minute                  0
dtype: int64

In [8]:
meta.player_name.value_counts()

player_name
E. Haaland        7071
R. Jiménez        3689
A. Isak           3526
D. Solanke        3282
N. Jackson        3143
                  ... 
C. Jones            42
E. Smith Rowe       37
M. Gibbs-White      26
E. Ferguson         22
A. Grønbæk          20
Name: count, Length: 87, dtype: int64

In [82]:
with fs.open("arthur_tmp/pitch_control/1650961.npz", "rb") as f:
    data = np.load(f, allow_pickle=True)
    results = data["results"].tolist()

In [83]:
frame = next((d for d in results if d["frame"] == 56795), None)

In [84]:
frame

{'pitch_control_map': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'defensive_lines': [27.199038461538464, 60.95552884615384, 88.21153846153845],
 'frame': 56795,
 'player_id': 2766,
 'player_position': (nan, nan),
 'ball_position': (99.68942307692309, 26.159999999999997),
 'in_possession': False,
 'distance_to_ball': nan,
 'distance_to_nearest_teammate': nan,
 'distance_to_nearest_opponent': nan,
 'player_position_role': 'Center Forward'}

In [14]:
len(frames_inputs)

73120

In [15]:
frames_inputs[0]

{'pitch_control_map': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'defensive_lines': [60.455769230769235, 71.61605769230769, 82.43762019230769],
 'frame': 1625,
 'player_id': 1498,
 'player_position': (77.88173076923077, 50.82),
 'ball_position': (68.65384615384616, 57.5),
 'in_possession': False,
 'distance_to_ball': 11.391938135139778,
 'distance_to_nearest_teammate': 10.26718897565972,
 'distance_to_nearest_opponent': 5.084613021681336,
 'player_position_role': 'Center Forward'}

In [16]:
umap_kwargs = dict(n_neighbors=200, min_dist=0.3, n_components=5, random_state=42)
hdbscan_kwargs = dict(min_cluster_size=6000, min_samples=50, cluster_selection_method='eom',
                      cluster_selection_epsilon=0.0, metric='euclidean')

In [ ]:
#umap_kwargs = dict(n_neighbors=100, min_dist=0.2, n_components=5, random_state=42)
#hdbscan_kwargs = dict(min_cluster_size=3000, min_samples=30, cluster_selection_method='eom')

In [ ]:
#umap_kwargs = dict(n_neighbors=50, min_dist=0.1, n_components=10, random_state=42)
#hdbscan_kwargs = dict(min_cluster_size=1500, min_samples=15, cluster_selection_method='leaf')

In [ ]:
%%time

# 1. Standardisation
X_scaled = StandardScaler().fit_transform(X)

# 2. PCA pour compresser à 50 dimensions
pca = PCA(n_components=50, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# 3. UMAP sur PCA
reducer = umap.UMAP(
    n_neighbors=100,   # structure locale
    min_dist=0.2,     # densité de projection
    n_components=20,   # pour clustering
    random_state=42
)
embedding = reducer.fit_transform(X_pca)

# 4. HDBSCAN
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=3000,
    min_samples=30,
    cluster_selection_epsilon=0.05,
    cluster_selection_method='leaf'
).fit(embedding)

labels = clusterer.labels_
n_clusters = len(set(labels[labels >= 0]))
print("clusters:", n_clusters, "noise:", np.sum(labels==-1))

IOStream.flush timed out


In [ ]:
#def cluster_pitch_control_with_context(X, n_neighbors=20, n_components=20, min_dist=0.1, min_cluster_size=30):
#    """
#    Applique UMAP + HDBSCAN sur les features combinées PitchControl + Context.
#    Retourne les embeddings UMAP et les labels HDBSCAN.
#    """
#    # ⚖️ Standardisation : utile ici car les context features sont sur des échelles différentes
#    X_scaled = StandardScaler().fit_transform(np.nan_to_num(X))
#
#    # 🌀 Réduction UMAP
#    reducer = umap.UMAP(
#        n_neighbors=n_neighbors,
#        min_dist=min_dist,
#        n_components=n_components,
#        metric="euclidean",
#        random_state=42
#    )
#    embedding = reducer.fit_transform(X_scaled)
#
#    # 🧩 Clustering HDBSCAN
#    clusterer = hdbscan.HDBSCAN(
#        min_cluster_size=min_cluster_size,
#        min_samples=None,
#        cluster_selection_epsilon=0.01,
#        cluster_selection_method='leaf',
#        metric='euclidean'
#    ).fit(embedding)
#
#    labels = clusterer.labels_
#    probs = clusterer.probabilities_
#
#    return embedding, labels, probs, reducer, clusterer

In [ ]:
#embedding, labels, probs, reducer, clusterer = cluster_pitch_control_with_context(
#    X=X,
#    n_neighbors=10,
#    n_components=40,
#    min_dist=0.05,
#    min_cluster_size=5
#)

In [ ]:
labels

In [ ]:
len(set(labels)) - (1 if -1 in labels else 0)  # nombre de clusters (excluant le bruit)

In [ ]:
meta["cluster"] = labels

In [ ]:
meta_path = f"{CLUSTERING_DIR}/meta_frames_center_forward_w_labels.parquet"
with fs.open(meta_path, "wb") as f:
    meta.to_parquet(f, index=False)

In [ ]:
def plot_cluster_summary_pitch_control_with_pitch(
    results,
    meta,
    n_cols=3,
    pitch_length=105,
    pitch_width=68,
    half_pitch=True
):
    """
    Affiche la carte moyenne de pitch control et les features contextuelles moyennes par cluster.
    """

    # Clusters valides (on exclut les outliers HDBSCAN = -1)
    unique_clusters = sorted([c for c in meta["cluster"].unique() if c != -1])
    if len(unique_clusters) == 0:
        print("Aucun cluster valide (hors -1) trouvé dans meta['cluster'].")
        return

    n_clusters = len(unique_clusters)
    n_rows = int(np.ceil(n_clusters / n_cols))

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 6 * n_rows))
    axes = axes.flatten()

    cmap = plt.cm.RdBu_r
    cmap.set_bad(color="white")

    pcm = None  # utilisé pour la colorbar globale

    for i, cluster_id in enumerate(unique_clusters):
        ax = axes[i]

        # ---- Extraire les éléments du cluster ----
        cluster_indices = meta.index[meta["cluster"] == cluster_id].tolist()
        cluster_items = [results[idx] for idx in cluster_indices]

        if len(cluster_items) == 0:
            ax.axis("off")
            continue

        # ---- Moyenne des cartes de pitch control (ignorer NaN) ----
        maps = np.array([item["pitch_control_map"] for item in cluster_items])
        # maps shape = (n_frames, H, W)
        mean_map = np.nanmean(maps, axis=0)  # si certains éléments contiennent NaN, on les ignore

        # ---- Moyenne des lignes défensives ----
        # On s'assure que chaque defensive_lines est un array de longueur 3 (avec NaN si manquant)
        def_lines_list = []
        for item in cluster_items:
            lines = np.array(item.get("defensive_lines", []), dtype=float)
            if lines.size == 0:
                padded = np.array([np.nan, np.nan, np.nan])
            else:
                padded = np.pad(lines, (0, max(0, 3 - len(lines))), mode="constant", constant_values=np.nan)
            def_lines_list.append(padded)
        all_lines = np.vstack(def_lines_list)  # shape (n_frames, 3)
        mean_lines = np.nanmean(all_lines, axis=0)  # résultat shape (3,)
        # si toutes les valeurs d'une colonne sont NaN, np.nanmean renvoie nan -> on détecte ensuite

        # ---- Moyennes des features contextuelles (ignorer NaN) ----
        # utils : colonne peut contenir NaN
        def nanmean_safe(lst):
            arr = np.array(lst, dtype=float)
            if np.all(np.isnan(arr)):
                return np.nan
            return float(np.nanmean(arr))

        mean_dist_ball = nanmean_safe([item.get("distance_to_ball", np.nan) for item in cluster_items])
        mean_dist_tm = nanmean_safe([item.get("distance_to_nearest_teammate", np.nan) for item in cluster_items])
        mean_dist_op = nanmean_safe([item.get("distance_to_nearest_opponent", np.nan) for item in cluster_items])

        # ---- Possession ratio & nombre de frames ----
        in_poss_array = np.array([1.0 if item.get("in_possession", False) else 0.0 for item in cluster_items], dtype=float)
        possession_ratio = float(np.nanmean(in_poss_array)) if in_poss_array.size > 0 else np.nan
        n_frames = len(cluster_items)

        # ---- Création du pitch + grille ----
        pitch = Pitch(
            pitch_type="custom",
            pitch_length=pitch_length,
            pitch_width=pitch_width,
            line_color="black",
            pitch_color="white"
        )

        H, W = mean_map.shape
        x = np.linspace(0, pitch_length, W)
        y = np.linspace(0, pitch_width, H)
        bin_statistic = dict(statistic=mean_map, x_grid=x, y_grid=y)

        # ---- Plot de la heatmap (utilise mean_map, NaN seront affichés en blanc grâce à cmap.set_bad) ----
        pitch.draw(ax=ax)
        pcm = pitch.heatmap(
            bin_statistic,
            ax=ax,
            cmap=cmap,
            vmin=0,
            vmax=1,
            alpha=0.9,
        )

        if half_pitch:
            ax.set_xlim(pitch_length / 2, pitch_length)
            ax.set_ylim(0, pitch_width)

        # ---- Lignes défensives : ne tracer que les composantes non-NaN ----
        if mean_lines.size > 0 and not np.all(np.isnan(mean_lines)):
            colors = ["black", "gray", "silver"]
            for j, x_mean in enumerate(mean_lines):
                if not np.isnan(x_mean):
                    ax.axvline(x=float(x_mean), color=colors[j % len(colors)], linestyle="--", linewidth=2)

        # ---- Titres & annotations ----
        # formatage des moyennes en remplaçant nan par 'n/a'
        def fmt(v):
            return f"{v:.2f}" if (v is not None and not np.isnan(v)) else "n/a"

        stats_text = (
            f"Frames: {n_frames}  |  Possession: {possession_ratio:.0%}\n"
            f"Dist. Ballon: {fmt(mean_dist_ball)} m  |  "
            f"Dist. Coéquipier: {fmt(mean_dist_tm)} m  |  "
            f"Dist. Adversaire: {fmt(mean_dist_op)} m"
        )

        ax.set_title(f"Cluster {cluster_id}", fontsize=13, fontweight="bold")
        ax.text(0.5, -0.08, stats_text, transform=ax.transAxes, fontsize=10, ha="center", va="top")

    # ---- Supprimer les axes vides (derniers) ----
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    # ---- Ajustement des marges pour éviter le chevauchement ----
    plt.subplots_adjust(right=0.92, wspace=0.35, hspace=0.45)

    # ---- Barre de couleur globale (placée à droite, hors de la zone des subplots) ----
    if pcm is not None:
        cbar_ax = fig.add_axes([0.94, 0.25, 0.02, 0.5])  # [left, bottom, width, height] en fraction de la fig
        cbar = fig.colorbar(pcm, cax=cbar_ax)
        cbar.set_label("Mean Pitch Control Probability", fontsize=11)

    fig.suptitle("Cartes moyennes de Pitch Control par Cluster", fontsize=18, fontweight="bold")
    plt.show()

In [ ]:
plot_cluster_summary_pitch_control_with_pitch(frames_inputs, meta, n_cols=3)

In [ ]:
def plot_representative_frames_for_cluster(
    cluster_id,
    X_embedded,
    meta,
    results,
    n_representatives=3,
    pitch_length=105,
    pitch_width=68,
    half_pitch=True
):
    """
    Affiche les n_representatives frames les plus proches du centroïde du cluster donné.
    Montre aussi les lignes défensives et les infos contextuelles sous chaque frame.
    
    Paramètres
    ----------
    cluster_id : int
        Identifiant du cluster à afficher.
    X_embedded : np.ndarray
        Embeddings (UMAP ou PCA) utilisés pour le clustering. shape=(n_frames, n_dim)
    meta : pd.DataFrame
        Métadonnées alignées avec X_embedded (doit contenir 'cluster').
    results : list
        Liste de dictionnaires correspondant à chaque frame.
    n_representatives : int
        Nombre de frames les plus représentatives à afficher.
    pitch_length, pitch_width : float
        Dimensions du terrain.
    half_pitch : bool
        Si True, n'affiche que la moitié offensive du terrain.
    """
    # --- Sélection des frames du cluster ---
    cluster_mask = (meta["cluster"] == cluster_id)
    cluster_indices = np.where(cluster_mask)[0]
    if len(cluster_indices) == 0:
        print(f"Aucun frame trouvé pour le cluster {cluster_id}.")
        return
    
    X_cluster = X_embedded[cluster_indices]

    # --- Approximation du centroïde (moyenne des embeddings du cluster) ---
    centroid = X_cluster.mean(axis=0, keepdims=True)

    # --- Distance à ce centroïde ---
    distances = pairwise_distances(X_cluster, centroid).flatten()
    top_indices = np.argsort(distances)[:n_representatives]
    representative_indices = cluster_indices[top_indices]

    # --- Setup du plot ---
    fig, axes = plt.subplots(1, n_representatives, figsize=(6 * n_representatives, 6))
    if n_representatives == 1:
        axes = [axes]

    pitch = Pitch(
        pitch_type="custom",
        pitch_length=pitch_length,
        pitch_width=pitch_width,
        line_color="black",
        pitch_color="white"
    )

    cmap = plt.cm.RdBu_r
    cmap.set_bad(color="white")

    # --- Boucle sur les frames représentatives ---
    for i, idx in enumerate(representative_indices):
        ax = axes[i]
        frame_data = results[idx]

        # --- Pitch control map ---
        pc_map = frame_data["pitch_control_map"]
        H, W = pc_map.shape
        x = np.linspace(0, pitch_length, W)
        y = np.linspace(0, pitch_width, H)
        bin_statistic = dict(statistic=pc_map, x_grid=x, y_grid=y)

        pitch.draw(ax=ax)
        pcm = pitch.heatmap(bin_statistic, ax=ax, cmap=cmap, vmin=0, vmax=1, alpha=0.9)

        if half_pitch:
            ax.set_xlim(pitch_length / 2, pitch_length)
            ax.set_ylim(0, pitch_width)

        # --- Lignes défensives ---
        def_lines = np.array(frame_data.get("defensive_lines", []), dtype=float)
        if def_lines.size > 0:
            colors = ["black", "gray", "silver"]
            for j, x_line in enumerate(def_lines[:3]):  # max 3 lignes
                ax.axvline(x=float(x_line), color=colors[j % len(colors)], linestyle="--", linewidth=2)

        # --- Informations contextuelles ---
        def fmt(v):
            return f"{v:.2f}" if v is not None and not np.isnan(v) else "n/a"

        dist_ball = frame_data.get("distance_to_ball", np.nan)
        dist_tm = frame_data.get("distance_to_nearest_teammate", np.nan)
        dist_op = frame_data.get("distance_to_nearest_opponent", np.nan)
        in_poss = "✅" if frame_data.get("in_possession", False) else "❌"

        # --- Infos depuis meta ---
        frame_id = meta.loc[idx, "frame"]
        time_min = meta.loc[idx, "time"]
        player = meta.loc[idx, "player_short_name"]
        team = meta.loc[idx, "team_name"]

        # --- Titre au-dessus et texte en-dessous ---
        ax.set_title(
            f"Cluster {cluster_id} | Frame {frame_id}\n{team} - {player}\nMinute {time_min}",
            fontsize=12, fontweight="bold"
        )

        stats_text = (
            f"Possession: {in_poss}  |  Dist. Ballon: {fmt(dist_ball)} m\n"
            f"Dist. Coéquipier: {fmt(dist_tm)} m  |  Dist. Adversaire: {fmt(dist_op)} m"
        )
        ax.text(0.5, -0.12, stats_text, transform=ax.transAxes,
                fontsize=10, ha="center", va="top", wrap=True)

    plt.suptitle(f"Frames les plus représentatives du cluster {cluster_id}", fontsize=16, fontweight="bold")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_representative_frames_for_cluster(
    cluster_id=3,
    X_embedded=embedding,
    meta=meta,
    results=frames_inputs,
    n_representatives=3
)